In [ ]:
Z# INSTALLINGS

!pip install tensorflow==2.1.0
!pip install keras==2.3.1
!pip install segmentation_models==1.0.1
!pip install h5py==2.10.0 
!pip install plotly==5.3.1
!pip install scikit-learn

     |████████████████████████████████| 421.8 MB 23 kB/s 
     |████████████████████████████████| 3.8 MB 34.5 MB/s 
     |████████████████████████████████| 448 kB 47.4 MB/s 
     |████████████████████████████████| 50 kB 7.0 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=c27e07c566d39d684b13195f4e394a8648b47bda804f310bc84b556030486c8b
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Succes

In [ ]:
# LINKAGE TO GOOGLE DRIVE AND LIBRERIES IMPORTING

from google.colab import drive
drive.mount('/content/drive')

import os
import numpy as np
import tensorflow as tf

from matplotlib import pyplot as plt
from tqdm import tqdm
from skimage.io import imread
from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from segmentation_models import Unet

Mounted at /content/drive


Using TensorFlow backend.


Segmentation Models: using `keras` framework.


In [ ]:
# DATASET UNRAR: LOADING DATASET IN COLAB

!pip install unrar
!unrar x "drive/MyDrive/cytology challenge/00_DATASET/train.rar"     # unraring training set
!unrar x "drive/MyDrive/cytology challenge/00_DATASET/validation.rar"   # unraring validation set
!unrar x "drive/MyDrive/cytology challenge/00_DATASET/test.rar"         # unraring test set


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from drive/MyDrive/cytology challenge/00_DATASET/train.rar

Creating    train                                                     OK
Creating    train/images                                              OK
Extracting  train/images/102.tif                                           0%  OK 
Extracting  train/images/104.tif                                           0%  1%  OK 
Extracting  train/images/106.tif                                           1%  OK 
Extracting  train/images/108.tif                                           1%  2%  OK 
Extracting  train/images/109.tif                                           2%  OK 
Extracting  train/images/111.tif                                           2%  3%  OK 
Extracting  train/images/112.tif                                           3%  OK 
Extracting  train/images/114.tif              

In [ ]:
# LOAD PRE-PROCESSED TRAINING SET AND VALIDATION SET (READY TO USE) and SET UP NET'S SETTINGS

# load
images = np.load('drive/MyDrive/EIM challenge_gruppo_FA_DO_PA_PA/01_PRE-PROCESSED FOR TRAINING/pre_proc.npz')  # images loading 
labels = np.load('drive/MyDrive/EIM challenge_gruppo_FA_DO_PA_PA/01_PRE-PROCESSED FOR TRAINING/manual_masks.npz')  # manual annotations loading 
X_tr = images['X_tr']
X_vl = images['X_vl']
Y_tr = labels['Y_tr']
Y_vl = labels['Y_vl']
images.close()
labels.close()
del images,labels

## class representation (in percentage) in training set 
pc_nuc = (Y_tr==2).sum()/Y_tr.size  # nucleus class
pc_cyto = (Y_tr==1).sum()/Y_tr.size  # cytoplasm class
pc_bg = (Y_tr==0).sum()/Y_tr.size  # background class
print(f"\nrepresentation of classes in TRAINING SET:   NUCLEUS {pc_nuc} | CYTOPLASM {pc_cyto} | BACKGROUND {pc_bg}")
del pc_nuc,pc_cyto,pc_bg

# settings
current_net = 'X'  # name of the model that will be trained <----------------------------------------------- CHANGE HERE
rsz = X_tr.shape[1]  # resizing size (resize images to rsz x rsz) 
NUM_CLASSES = Y_tr.max() + 1 # number of classes choosen for the problem 

# conversion of labels to categorical: to minimize required space for storage 
# labels has been saved as greyscale images and not as categorical labels, 
# conversion is done here
Y_tr = to_categorical(Y_tr, num_classes = NUM_CLASSES, dtype='float32')  # conversion to categorical data
Y_vl = to_categorical(Y_vl, num_classes = NUM_CLASSES, dtype='float32')  # conversion to categorical data


representation of classes in TRAINING SET:   NUCLEUS 0.0 | CYTOPLASM 0.3333333333333333 | BACKGROUND 0.6666666666666666


In [ ]:
# DATA AUGMENTATION SETTINGS 

# color augmentation function 
def coloraug(image):
      
      if image.dtype == 'float32':
        image = image
      elif image.dtype == 'uint8': 
        image = tf.image.random_hue(image,0.05)
        image = tf.image.random_saturation(image,0.95,1.05)
        image = tf.image.random_contrast(image,0.95,1.05)
      return image

# training set augmentation (comment all parameters setting lines to not perform
# data augmentation)
image_datagen = ImageDataGenerator(            
                                  horizontal_flip = True,
                                  vertical_flip = True,
                                  fill_mode = 'constant',
                                  cval = 0,
                                  zoom_range = [0.8,1.2],
                                  # brightness_range=[0.9,1.5],
                                  # preprocessing_function = coloraug
                                  )

# validation set augmentation
val_datagen = ImageDataGenerator()

# Generator function
def XYaugmentGenerator(x, y, batch_sz, sd):
        if sd == 'rnd':
          sd = np.random.randint(1,9999)
        genX = image_datagen.flow(x, batch_size=batch_sz,seed = sd)
        genY = image_datagen.flow(y, batch_size=batch_sz, seed = sd)
        while True:
            Xi = genX.next()  #The next() function returns the next item in an iterator.
            Yi = genY.next()
            yield Xi, Yi

In [ ]:
# NET TRAINING 

# defining U-NET architecture
BACKBONE = 'densenet201'
model = Unet(backbone_name = BACKBONE,
            input_shape = (rsz,rsz,3),
            encoder_weights = 'imagenet', 
            encoder_freeze = True,
            decoder_block_type = 'transpose',
            classes = NUM_CLASSES,
            activation = 'softmax')

# defining optimization algorithm and loss function
model.compile('Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# training and parameters <---------------------------------------------------------------------- CHANGE HERE (below)
n_epochs = 20
batch_sz = 2
sd = 1
monitor_value = 'val_categorical_accuracy'
verbose_value = 1
min_delta_value = 0.001
patience_value = 10
mode_value = 'max'

# checkpoint definition
csv_logger = CSVLogger('./log.out', append=True, separator=';')
earlystopping = EarlyStopping(monitor = monitor_value, verbose = verbose_value, min_delta = min_delta_value, patience = patience_value, mode = mode_value, restore_best_weights = True)
callbacks_list = [csv_logger, earlystopping]




# model training
results = model.fit(XYaugmentGenerator(X_tr,Y_tr, batch_sz, sd), 
                    steps_per_epoch = np.ceil(float(len(X_tr))/float(batch_sz)),
                    validation_data = val_datagen.flow(X_vl,Y_vl,batch_sz), 
                    validation_steps = np.ceil(float(len(X_vl))/float(batch_sz)),
                    shuffle = True,
                    epochs = n_epochs,
                    callbacks = callbacks_list) 

74842112/74836368 [==============================] - 5s 0us/step
Epoch 1/20
150/150 [==============================] - 260s 2s/step - loss: 0.2928 - categorical_accuracy: 0.8661 - val_loss: 0.1435 - val_categorical_accuracy: 0.9325
Epoch 2/20
150/150 [==============================] - 162s 1s/step - loss: 0.1689 - categorical_accuracy: 0.9196 - val_loss: 0.0903 - val_categorical_accuracy: 0.9468
Epoch 3/20
150/150 [==============================] - 162s 1s/step - loss: 0.1543 - categorical_accuracy: 0.9171 - val_loss: 0.0607 - val_categorical_accuracy: 0.9532
Epoch 4/20
150/150 [==============================] - 162s 1s/step - loss: 0.1426 - categorical_accuracy: 0.9154 - val_loss: 0.0781 - val_categorical_accuracy: 0.9544
Epoch 5/20
150/150 [==============================] - 162s 1s/step - loss: 0.1408 - categorical_accuracy: 0.9265 - val_loss: 0.1525 - val_categorical_accuracy: 0.9424
Epoch 6/20
150/150 [==============================] - 161s 1s/step - loss: 0.1283 - categorical_accu

In [ ]:
# TRAINED MODEL SAVING

if not os.path.exists('drive/MyDrive/EIM challenge_gruppo_FA_DO_PA_PA/02_TRAINED MODELS/'):
    os.mkdir('drive/MyDrive/EIM challenge_gruppo_FA_DO_PA_PA/02_TRAINED MODELS/')

model.save('drive/MyDrive/EIM challenge_gruppo_FA_DO_PA_PA/02_TRAINED MODELS/' + current_net)